# Interactive Visual Analytics with Folium

In [2]:
import pandas as pd
import folium as fl
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

### Here 'dataset_part_2.csv' can be used to extract the required columns into another temp dataframe to utilize for this notebook.

### Load data

In [3]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(url)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


### We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map=fl.Map(location=nasa_coordinate,zoom_start=4)

### We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. 

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = fl.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(fl.Popup('NASA Johnson Space Center🚀'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = fl.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

### Do the same with the 4 launch centers.


In [7]:
coor1=[28.562302,-80.577356]
coor2=[28.563197,-80.576820]
coor3=[28.573255,-80.646895]
coor4=[34.632834,-120.610745]
c1=fl.Circle(coor1, radius=1000, color="#D42929", fill=True).add_child(fl.Popup('CCAFS LC-40'))
m1=fl.map.Marker(coor1, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40', ))
c2=fl.Circle(coor2, radius=1000, color="#C52121", fill=True).add_child(fl.Popup('CCAFS SLC-40'))
m2=m1=fl.map.Marker(coor2, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40', ))
c3=fl.Circle(coor3, radius=1000, color="#CC2121", fill=True).add_child(fl.Popup('KSC LC-39-A'))
m3=m1=fl.map.Marker(coor3, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A', ))
c4=fl.Circle(coor4, radius=1000, color="#C32525", fill=True).add_child(fl.Popup('VAFB SLC-4E'))
m4=m1=fl.map.Marker(coor4, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E', ))
site_map.add_child(c1)
site_map.add_child(c2)
site_map.add_child(c3)
site_map.add_child(c4)
site_map.add_child(m1)
site_map.add_child(m2)
site_map.add_child(m3)
site_map.add_child(m4)


### Mark the success/failed launches for each site on the map

In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [9]:
marker_cluster=MarkerCluster()


In [10]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
site_map.add_child(marker_cluster)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [11]:
# Iterate through the DataFrame and add markers # AI💀😑
for index, record in spacex_df.iterrows():
    # Extract coordinates
    lat = record['Lat']
    lon = record['Long']
    
    # Customize icon color based on success
    icon_color = record['marker_color']  # e.g., 'green' for success, 'red' for failure
    
    # Create marker
    marker = fl.Marker(
        location=[lat, lon],
        icon=fl.Icon(color='white', icon_color=icon_color),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {record['class']}"
    )
    
    # Add marker to cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

### Calculate the distances between a launch site to its proximities

In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [14]:
# Coordinates
launch_site_lat = 28.56357
launch_site_lon = -80.57678
coastline_lat = 28.56367
coastline_lon = -80.56759

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Add coastline marker
fl.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline',
    icon=fl.Icon(color='green')
).add_to(site_map)

# Add distance label using DivIcon
fl.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline)
    )
).add_to(site_map)

# Draw PolyLine between launch site and coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = fl.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)


# END